
## Bibliotecas Usadas



In [1]:
import random
import copy
import math
#import tqdm
from tqdm import tqdm_notebook as tqdm
import numpy as np
import heapq
from IPython.core.display import display, HTML


## Funções para o tabuleiro

In [2]:
def printf(jogo):
    string = ""
    
    pos = list(enumerate(jogo))
    
    dimensao = len(jogo)
    
    black = '<span style="background-color:black;color:white;'
    white = '<span style="background-color:white;color:black;'
    
    add = 'padding:0;margin:0;border:0;font-size:17px;">'
    
    for i in range(dimensao):
        for j in range(dimensao):
            if (i%2 == 0 and j%2 == 0) or (i%2 != 0 and j%2 != 0): string += black + add
            else: string += white + add
    
            if (j, i) in pos: string += "&nbsp;♛&nbsp;"
            else: string += "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
            
            string += "</span>"
        string += "<br>"
    
    display(HTML(string))

In [4]:
def heuristica(jogo):
    
    #peso do tabuleiro
    peso = 0

    #conjunto de pecas nao verificadas
    #partes_ocupadas = list(jogo.pos_damas.keys())
    
    dimensao = len(jogo)
    
    for i in range(dimensao):
        
        for j in range(i+1, dimensao):
            
            posi = np.array([i,jogo[i]])
            
            posj = np.array([j, jogo[j]])
            
            delta = abs(posi - posj)
            
            if delta[0] == delta[1] or posi[0] == posj[0] or posi[1] == posj[1]:
                peso += 1
        
    return peso    

In [5]:
def movimentacaoRandomica(jogo):
    
    novo_jogo = copy.deepcopy(jogo)
    
    dimensao = len(jogo)
    
    #peca aleatoria
    peca = random.randint(0, dimensao-1)
    
    escolha = random.randint(0, dimensao-1)
    
    novo_jogo[peca] = escolha
    
    return novo_jogo

In [6]:
def movimentacaoRandomicaInteligente(jogo):
    
    novo_jogo = copy.deepcopy(jogo)
    
    dimensao = len(jogo)
    
    #peca aleatoria
    coluna1 = random.randint(0, dimensao-1)
    
    coluna2 = random.randint(0, dimensao-1)
    
    linha2 = novo_jogo[coluna2]
    novo_jogo[coluna2] = novo_jogo[coluna1]
    novo_jogo[coluna1] = linha2
    
    return novo_jogo

In [7]:
def tabRandomico(dim):
  
  coiso = random.sample(range(dim), k = dim)
  
  tab = np.array(coiso)
  
  return tab

## Simulated Anneling

In [8]:
def simulatedAnnealing(funcao_peso, funcao_jogada, jogo, MAX = 10000, C=1, passado=None):
    
    
    if isinstance(passado, type(None)): 
        atual = copy.deepcopy(jogo)
    else:
        atual = passado
    
    prob = lambda x, y, z: np.exp( (x - y)/z )
    
    t = np.arange(1, MAX)
    
    T = C/np.sqrt(t) 
    
    pbar = tqdm(total=MAX)

    for t in T:
                    
        viz = funcao_jogada(atual)
        
        p_atual = funcao_peso(atual)
        p_viz = funcao_peso(viz)
        
        if t <= 0.005 or p_atual == 0: return atual
        
        if p_viz <= p_atual:
            atual = viz
        
        else:
            probabilidade = prob(p_atual, p_viz, t)
            atual = random.choices([viz, atual], weights= [probabilidade, 1-probabilidade])[0]
        
        #pbar.update(1)
        pbar.set_description("Peso atual: {}".format( funcao_peso(atual) ) )
        pbar.update(1)

    while True:
        resposta = input('O peso atual é {}, gostaria de continuar? [y/n] '.format(funcao_peso(atual)))
        
        if resposta == 'y': 
            return simulatedAnnealing(funcao_peso, funcao_jogada, jogo,C=C,MAX=MAX, passado=atual)
        elif resposta == 'n':
            return atual

In [19]:
jogo = np.arange(10)
print("Quantidade de ataques do tabuleiro inicial:", heuristica(jogo))

Quantidade de ataques do tabuleiro inicial: 45


In [20]:
resposta = simulatedAnnealing(heuristica,
                  movimentacaoRandomicaInteligente,
                  jogo)


In [21]:
printf(resposta)
print("Quantidade de ataques da resposta encontrada:", heuristica(resposta))

Quantidade de ataques da resposta encontrada: 0


## Algoritmo Genetico

In [23]:
def cruzamento(pai, mae):
    
    dimensao = len(pai)
    
    crossouver = random.randint(1,len(pai)-1)
    
    f1 = [*pai[:crossouver], *mae[crossouver:]]
        
    return f1

In [24]:
def cruzamentoIcaro(pai, mae):
  
  f1 = np.arange(len(pai))
  
  alternado = lambda x: mae[x] if x%2==0 else pai[x]
  
  return list(map(alternado, f1))

In [25]:
def cruzamentoInteligente(pai, mae):

    dimensao = len(pai)
    
    f1 = cruzamentoIcaro(pai, mae)
    
    set_filho = set(f1)
    sub = set(pai) - set_filho
    
    for i in range(dimensao):
        aux = f1[i]
        if aux in set_filho:
            set_filho.remove(aux)
        else:
            f1[i] = sub.pop()
    
    return f1

In [26]:
def mutacao(jogo):
    
    dimensao = len(jogo)
    
    pecas = random.sample(range(dimensao), 2)
    
    jogo[pecas[1]] = pecas[0]

In [31]:
def mutacaoInteligente(jogo):
    pass

In [27]:
def algoritmoGenetico(populacao, f_heuristica, f_cruzamento, f_mutacao, MAX = 1000, passado = None):
    
    def procriacao(pais):
        
        pai, mae = pais
        f1 = f_cruzamento(pai, mae)

        mutacao = random.choices([True, False], weights=[0.1, 0.9])[0]

        if mutacao: f_mutacao(f1)
          
        return (f_heuristica(f1), f1)
          
    tam = len(populacao)
    
    chave = lambda x: x[0]
    
    pop_sobrev = round(0.2*tam)
    
    escolha = lambda x, y: random.choices(y[:,1], weights = 1/y[:,0], k = 2)
    
    if isinstance(passado, type(None)):
        pop = copy.deepcopy(populacao)
      
        p_map = list(map(lambda x: (f_heuristica(x),x) , pop))
      
        ordem = np.array(sorted(p_map, key = chave))
    
    else:
        ordem = passado
      
      
    pbar = tqdm(total = MAX)
    
    for iter in range(MAX):
        
        nova_pop = heapq.nsmallest(pop_sobrev, ordem, key=chave)
        
        if nova_pop[0][0] == 0: return nova_pop[0][1]
        
        filhos = list(map(procriacao, map(lambda x: escolha(x,ordem), range(round(0.8*tam)))))
        
        ordem = np.array([*nova_pop, *filhos])
        
        pbar.set_description("Melhor peso: {}".format(nova_pop[0][0]))
        pbar.update(1)
    
    minimo = min(ordem, key=chave)
    while True:
        
        resposta = input('O melhor individuo tem peso {}, quer continuar? [y/n]\n'.format(minimo[0]))

        if resposta == 'y':
            return algoritmoGenetico(populacao, f_heuristica, f_cruzamento, f_mutacao,MAX=MAX, passado = ordem)
        
        elif resposta == 'n': 
            return minimo[1]
    
            

In [32]:
pop = np.array([tabRandomico(40) for x in range(10)])

peso_map = list(map(lambda x: (heuristica(x),x) , pop))

ordem = np.array(sorted(peso_map, key = lambda x: x[0]))

ordem[0][0]

19

In [ ]:

resposta = algoritmoGenetico(pop,
                 heuristica,
                 cruzamentoInteligente,
                 mutacaoInteligente)


O melhor individuo tem peso 13, quer continuar? [y/n]
y


In [ ]:
printf(resposta)

## Colonia de Formigas

In [ ]:
def percorre(jogo)

In [ ]:

class ColoniaFormigas:
    def __init__(self, dimensao, quantidade_formigas):
        self.dimensao = dimensao
        self.quantidade_formigas = quantidade_formigas
        
        self.ro = 0.001  # (1 - ro) evaporação
        self.Q  = 1.0    # Quantidade de feromônio distribuído
        self.alfa = 10.0 # peso do feromônio
        self.beta = 1.0  # peso do peso da aresta
        
        self.formigas  = []
        self.feromonio = [[{'f':1000, 'p':1} for i in range(dimensao)] for j in range(dimensao) ]
        self.calcula_feromonio()
        self.max = 1000
    
    def percorre(self):
        dimensao = self.dimensao
        jogo = Jogo(dimensao)
        linhas_possiveis = [i for i in range(dimensao)]
        colunas = random.sample(range(dimensao), k=dimensao)

        for coluna in colunas:
            p = self.calcula_probabilidade(coluna, linhas_possiveis)
            linha = int(choice(linhas_possiveis, 1, p=p)[0])
            linhas_possiveis.remove(linha)
            jogo[coluna] = (linha, coluna)

        self.formigas.append({'jogo':jogo, 'peso':heuristicaIcaro(jogo)})
    
    
    #consulta! nao altera nd
    def calcula_probabilidade(self, linha, linhas_possiveis):
        p = []
        soma = 0.0
    
        for indice in linhas_possiveis:
            
            fero = self.feromonio[linha][indice]['f']
            peso = self.feromonio[linha][indice]['p']
            
            tudo = (fero**self.alfa)*(peso**self.beta)
            
            p.append(tudo)
            soma += (tudo)
        
        p = np.array([x for x in p])
        return p/soma
        
    def calcula_feromonio(self):
        dimensao = self.dimensao
       
        for i in range(dimensao):
            for j in range(dimensao):
                self.feromonio[i][j]['f'] = (1 - self.ro) * self.feromonio[i][j]['f']

                for formiga in self.formigas:
                    if (i, j) in formiga['jogo'].pos_damas.values():
                        self.feromonio[i][j]['f'] += self.Q/formiga['peso']

    def calcular(self):
        dimensao = self.dimensao
        
        for i in tqdm(range(1, self.max)):
            self.formigas = []
            threads = []
            
            if i % 500 == 0:
                self.Q += 1
            
            for i in range(self.quantidade_formigas):
                t = Thread(target=self.percorre())
                t.start()
                threads.append(t)
                
            for t in threads:
                t.join()
            
            for formiga in self.formigas: 
                if formiga['peso'] == 0: return formiga

            self.calcula_feromonio()
        
        menor = self.formigas[0]
        for formiga in self.formigas:
            if formiga['peso'] < menor['peso']: menor = formiga
                
        return menor